In [1]:
import numpy as np
import torch
import torch.nn as nn

In [32]:
def lacunarity_global(x, kernel_size = [3,3], eps = 10e-6):
    '''
    Lacunarity definition from Fast Unsupervised Seafloor Characterization
    in Sonar Imagery Using Lacunarity
    '''
    mean = np.mean(x)
    std = np.std(x)
    
    # #Lacunarity is the ratio between variance and squared mean
    L = ((std**2) / ((mean ** 2) + eps)) -1
    
    #Revisist Pytorch implementation
    # #Convert input to tensor
    x_tensor = torch.tensor(x).unsqueeze(0).unsqueeze(0).float()
    squared_x_tensor = x_tensor ** 2
    n_pts = np.prod(np.asarray(x_tensor.shape))
    #Define sum pooling 
    gap_layer = nn.AdaptiveAvgPool2d(1)(x_tensor)
    
    #Compute numerator (n * sum of squared pixels) and denominator (squared sum of pixels)
    L_numerator = ((n_pts)**2) * (nn.AdaptiveAvgPool2d(1)(squared_x_tensor))
    L_denominator = (n_pts * gap_layer)**2

    #Lacunarity is L_numerator / L_denominator - 1
    L_torch = (L_numerator / L_denominator) - 1
    
    # Convert back to numpy array
    L_torch = L_torch.squeeze(0).squeeze(0)
    L_numpy = L_torch.detach().cpu().numpy()

    return L_numpy

In [33]:
lacunarity_global([8, 3, 25, 25])

array([0.4221983], dtype=float32)